from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import sql
from pyspark.sql import SQLContext



#conf = SparkConf().setMaster("local[20]").setAppName("hw")
#sc = SparkContext(conf = conf)
sc = SparkContext()
sqlContext = sql.SQLContext(sc)

In [ ]:
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").load("/data/MSA_8050_Spring_18/proj09/final_merged_data.csv")

In [ ]:
li=['business_name',
 'neighborhood',
 'address','postal_code',
 'latitude',
 'longitude','is_open',
 'categories','review_date','yelping_since',
 'friends',
 'user_useful',
 'user_funny',
 'user_cool',
 'fans',
 'elite', 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

In [ ]:
for i in li:
    df=df.drop(i)

In [ ]:
df.columns

In [ ]:
df.select(['bus_business_id','review_count1']).show()

In [ ]:
df = df.withColumnRenamed("review_count1","bus_review_count")

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
from pyspark.sql.functions import sum
from pyspark.sql.functions import *

In [ ]:
df2=df.groupBy('user_id').agg(sum('review_count').alias('Total_Review')).sort(col('Total_Review').desc()).take(20)

In [ ]:
li2=[]
for i in range(20):
    li2.append(df2[i][0].encode("utf-8"))

In [ ]:
li2

In [ ]:
df_top_user=df.where(df.user_id.isin(li2))

In [ ]:
rdd_top_user=df_top_user.rdd.map(list)

In [ ]:
rdd_top_user.take(5)

In [ ]:
from collections import OrderedDict
import string
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")

In [ ]:
words =  rdd_top_user.map(lambda x:(x[-3], x[9].encode("utf-8").lower().translate(None,string.punctuation).strip()))


In [ ]:
words.take(5)

In [ ]:
stopwrds = set(stopwords.words('english'))


In [ ]:
X = words.reduceByKey(lambda x,y:x+y).map(lambda x:(x[0],x[1].split(" "))).map(lambda x:(x[0],{i:x[1].count(i) for i in set(x[1])}))


In [ ]:
Z = X.map(lambda x:(x[0],OrderedDict(sorted(x[1].items(), key = lambda y:y[1], reverse = True))))


In [ ]:
top_11 = Z.map(lambda x: (x[0], [i for i in x[1] if i not in stopwrds][:11]))


In [ ]:
for k,v in top_11.collect():
    print '{0} : {1}'.format(k,list(v)[1:11])

In [ ]:
df.count()